In [5]:
import json
import os
import pathlib

In [6]:
import json


def extract_data_to_json(file_name: str, file_path: str):
    datas = []
    sentences = []  # 存储每个句子

    # 读取文件并按行处理
    with open(file_path, 'r', encoding='utf-8') as file:
        current_sentence = []
        for line in file:
            line = line.strip()
            if line == '':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []  # 新的一句
            else:
                current_sentence.append(line)
        if current_sentence:
            sentences.append(current_sentence)  # 添加最后一句

    # 处理每个句子
    json_data = []
    for sentence in sentences:
        sentence_text = ' '.join([s.split(' ')[0] for s in sentence])  # 拼接出完整的句子
        datas = []

        # 解析句子中的标签
        for line in sentence:
            parts = line.split(' ')
            if len(parts) == 2:
                word, label = parts
                datas.append((word, label))

        entities = []
        current_entity = []
        current_label = None

        for word, label in datas:
            if label.startswith('B-'):  # B-开头表示实体的开始
                if current_entity:
                    entities.append((''.join(current_entity), current_label))  # 处理上一个实体
                current_entity = [word]
                current_label = label[2:]  # 获取实体标签
            elif label.startswith('I-') and current_entity:  # I-表示实体的内部部分
                current_entity.append(word)
            elif label == 'O' and current_entity:  # O表示没有实体
                entities.append((''.join(current_entity), current_label))
                current_entity = []
                current_label = None

        if current_entity:
            entities.append((''.join(current_entity), current_label))

        # 创建target字符串
        target_str = ""
        for label, entity in entities:
            target_str += f"{entity}实体：{label}\n"

        # 定义所有实体选项
        answer_choices = ["疾病", "医学检验项目", "医院科室", "身体部位", "微生物类", "临床表现", "药物", "西医治疗", "中医诊断", "西医诊断"]

        json_data.append({
            "input": sentence_text.replace(' ',''),
            "target": target_str.strip(),
            "answer_choices": answer_choices
        })

    # 将结果写入文件
    with open(file_name, 'w', encoding='utf-8') as file:
        json.dump(json_data, file, ensure_ascii=False, indent=4)


In [7]:
def extract_all_labels(file_path: str):
    all_labels = set()  # 用于存储所有标签

    # 读取文件并按行处理
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:  # 忽略空行
                parts = line.split(' ')
                if len(parts) == 2:
                    word, label = parts
                    if label != 'O':  # 只处理非'O'标签
                        all_labels.add(label[2:])  # 去掉'B-'或'I-'前缀，只保留标签名

    return list(all_labels)

In [8]:
directory = "./medician"
save_dir = "./medician_json"
data_map = {path_name.split(".")[-1] + ".json": os.path.abspath(os.path.join(directory, path_name)) for path_name in
            os.listdir(directory)}
print(data_map)

for file_name, abspath in data_map.items():
    extract_data_to_json(os.path.join(os.path.abspath(save_dir), file_name), abspath)

{'dev.json': 'D:\\pycharm_project\\CCKS2023-PromptCBLUE\\dataset\\medician\\medical.dev', 'test.json': 'D:\\pycharm_project\\CCKS2023-PromptCBLUE\\dataset\\medician\\medical.test', 'train.json': 'D:\\pycharm_project\\CCKS2023-PromptCBLUE\\dataset\\medician\\medical.train'}
